In [3]:
import os

os.environ['KALDI_ROOT'] = '/opt/kaldi/'

import kaldi_io
import numpy as np
import librosa
import IPython.display as ipd
import random
import kaldiio
from kaldiio import WriteHelper
from utils import add_noise
from numba.core.decorators import jit

In [4]:
from tqdm import tqdm

clean_train_folder = '../data/train/'
clean_test_folder = '../data/test/'
serialized_train_folder = '../data/serialized_train_data/'
serialized_test_folder = '../data/serialized_test_data/'
window_size = 2 ** 14  # about 1 second of samples
sample_rate = 16000

In [26]:
def slice_signal(wav, window_size, stride, sample_rate):
    """
    Helper function for slicing the audio file
    by window size and sample rate with [1-stride] percent overlap (default 50%).
    """
    #wav, sr = librosa.load(file, sr=sample_rate)
    hop = int(window_size * stride)
    slices = []
    for end_idx in range(window_size, len(wav), hop):
        start_idx = end_idx - window_size
        slice_sig = wav[start_idx:end_idx]
        slices.append(slice_sig)
    return slices

In [6]:
def process_and_serialize(data_type):
    """
    Serialize, down-sample the sliced signals and save on separate folder.
    """
    stride = 0.5

    if data_type == 'train':
        clean_folder = clean_train_folder
        #noisy_folder = noisy_train_folder
        serialized_folder = serialized_train_folder
    else:
        clean_folder = clean_test_folder
        #noisy_folder = noisy_test_folder
        serialized_folder = serialized_test_folder
    if not os.path.exists(serialized_folder):
        os.makedirs(serialized_folder)

    # walk through the path, slice the audio file, and save the serialized result
    for files in os.path.join(clean_folder, 'wav.scp'):
        for filename in tqdm(files, desc='Serialize and down-sample {} audios'.format(data_type)):
            clean_file = os.path.join(clean_folder, filename)
            noisy_file = os.path.join(noisy_folder, filename)
            # slice both clean signal and noisy signal
            clean_sliced = slice_signal(clean_file, window_size, stride, sample_rate)
            noisy_sliced = slice_signal(noisy_file, window_size, stride, sample_rate)
            # serialize - file format goes [original_file]_[slice_number].npy
            # ex) p293_154.wav_5.npy denotes 5th slice of p293_154.wav file
            for idx, slice_tuple in enumerate(zip(clean_sliced, noisy_sliced)):
                pair = np.array([slice_tuple[0], slice_tuple[1]])
                np.save(os.path.join(serialized_folder, '{}_{}'.format(filename, idx)), arr=pair)


In [31]:
def process_and_serialize(data_type):
    """
    Serialize, down-sample, augment the sliced signals and save on separate folder.
    
    """
    noise_choice = {'music':659, 'noise':929, 'speech':425}
    train_augmentation = 1
    stride = .5
    
    if data_type == 'train':
        clean_folder = clean_train_folder
        serialized_folder = serialized_train_folder
    else:
        clean_folder = clean_test_folder
        serialized_folder = serialized_test_folder
        
    if not os.path.exists(serialized_folder):
        os.makedirs(serialized_folder)

    
    for line in tqdm(open(os.path.join(clean_train_folder, 'wav.scp')),
                     desc='Serialize and down-sample {} audios'.format(data_type)):
        
        utt, path = line.rstrip().split()
        clean_file, _ = librosa.load(path, sr = 16000)
        for noise_type in noise_choice:
            for aug in range(train_augmentation):
                noise_track = np.random.randint(0, noise_choice[noise_type])
                _, noise_path = open('../data/musan_{}.scp'.format(noise_type)).readlines()[noise_track].rstrip().split()
                noise_audio, _ = librosa.load(noise_path, sr = 16000)
                noisy_file = add_noise(clean_file, noise_audio, snr=random.choice([2.5, 7.5, 12.5, 17.5]))

                # slice both clean signal and noisy signal
                clean_sliced = slice_signal(clean_file, window_size, stride, sample_rate)
                noisy_sliced = slice_signal(noisy_file, window_size, stride, sample_rate)
                # serialize - file format goes [original_file]_[noise_type]_[aug]_[slice_number].npy
                # ex) EN_C1_12_107.wav_0_5.npy denotes 5th slice of EN_C1_12_107.wav file in his augmentation version number 0
                for idx, slice_tuple in enumerate(zip(clean_sliced, noisy_sliced)):
                    pair = np.array([slice_tuple[0], slice_tuple[1]])
                    print('{}.wav_{}_{}_{}'.format(utt, noise_type, aug, idx))
                    #np.save(os.path.join(serialized_folder, '{}.wav_{}_{}'.format(utt, idx, aug)), arr=pair)
        break


In [32]:
process_and_serialize('train')

Serialize and down-sample train audios: 0it [00:00, ?it/s]

EN_C1_12_107.wav_music_0_0
EN_C1_12_107.wav_music_0_1
EN_C1_12_107.wav_music_0_2
EN_C1_12_107.wav_music_0_3
EN_C1_12_107.wav_music_0_4
EN_C1_12_107.wav_music_0_5
EN_C1_12_107.wav_music_0_6
EN_C1_12_107.wav_music_0_7
EN_C1_12_107.wav_music_0_8
EN_C1_12_107.wav_music_0_9
EN_C1_12_107.wav_music_0_10
EN_C1_12_107.wav_music_0_11
EN_C1_12_107.wav_noise_0_0
EN_C1_12_107.wav_noise_0_1
EN_C1_12_107.wav_noise_0_2
EN_C1_12_107.wav_noise_0_3
EN_C1_12_107.wav_noise_0_4
EN_C1_12_107.wav_noise_0_5
EN_C1_12_107.wav_noise_0_6
EN_C1_12_107.wav_noise_0_7
EN_C1_12_107.wav_noise_0_8
EN_C1_12_107.wav_noise_0_9
EN_C1_12_107.wav_noise_0_10
EN_C1_12_107.wav_noise_0_11
EN_C1_12_107.wav_speech_0_0
EN_C1_12_107.wav_speech_0_1
EN_C1_12_107.wav_speech_0_2
EN_C1_12_107.wav_speech_0_3
EN_C1_12_107.wav_speech_0_4
EN_C1_12_107.wav_speech_0_5
EN_C1_12_107.wav_speech_0_6
EN_C1_12_107.wav_speech_0_7
EN_C1_12_107.wav_speech_0_8
EN_C1_12_107.wav_speech_0_9
EN_C1_12_107.wav_speech_0_10
EN_C1_12_107.wav_speech_0_11
